In [2]:
from github import Github
from collections import Counter, defaultdict
from pydriller import Repository
import os
import git  
import shutil
import matplotlib.pyplot as plt
import os

## **Achando estatistícas das top 5 linguagens presentes no perfil do dev, seus respectivos repositórios e numero de commits relacionados a cada um.**

In [ ]:
GITHUB_TOKEN = "my token"
USERNAME = "enzoavalentim"

g = Github(GITHUB_TOKEN)    
user = g.get_user(USERNAME)

print(f"\n📛 Nome: {user.name}")
print(f"🔐 Login: {user.login}")
print(f"🖼️ Foto de perfil: {user.avatar_url}\n")

fotoPerfil = user.avatar_url
languageCommitCounter = Counter()
reposLanguage = defaultdict(list)

for repo in user.get_repos():
    try:
        commits = repo.get_commits()
        commit_count = commits.totalCount
        langs = repo.get_languages()
        repo_name = repo.name
        repo_url = repo.clone_url

        for lang in langs:
            languageCommitCounter[lang] += commit_count
            reposLanguage[lang].append({
                'name': repo_name,
                'commits': commit_count,
                'locAdd': 0,        
                'locRemov': 0,
                'url': repo_url     
            })

    except Exception as e:
        print(f"⚠️ Erro ao acessar {repo.name}: {e}")

top_5_languages = languageCommitCounter.most_common(5)
top5ReposLanguage = {lang: reposLanguage[lang] for lang, _ in top_5_languages}

print("\n📊 Top 5 linguagens mais ativas (por número de commits):")
for lang, total_commits in top_5_languages:
    print(f"➡️ {lang}: {total_commits} commits")

print("\n📂 Repositórios por linguagem (Top 5):")
for lang, repos in top5ReposLanguage.items():
    print(f"\n🗂️ {lang}:")
    for repo in repos:
        print(f"   - {repo['name']}: {repo['commits']} commits, +{repo['locAdd']} / -{repo['locRemov']} LOC")



📛 Nome: Enzo Augusto Valentim
🔐 Login: enzoavalentim
🖼️ Foto de perfil: https://avatars.githubusercontent.com/u/163206784?v=4


📊 Top 5 linguagens mais ativas (por número de commits):
➡️ Jupyter Notebook: 8 commits
➡️ Python: 3 commits
➡️ Java: 2 commits
➡️ C++: 1 commits
➡️ C: 1 commits

📂 Repositórios por linguagem (Top 5):

🗂️ Jupyter Notebook:
   - Analise-de-dados-agricolas-Disciplina-Estatistica-2024-02: 2 commits, +0 / -0 LOC
   - CurriculoGitHub: 6 commits, +0 / -0 LOC

🗂️ Python:
   - Estudos-Iniciais-Em-Python: 2 commits, +0 / -0 LOC
   - Trabalhos-Praticos-Programacao-de-Computadores-e-Resolucao-de-Problemas---Python-2023-02: 1 commits, +0 / -0 LOC

🗂️ Java:
   - Trabalhos-POO-2024-01: 2 commits, +0 / -0 LOC

🗂️ C++:
   - Estrurua-de-dados-2024-02: 1 commits, +0 / -0 LOC

🗂️ C:
   - Estrurua-de-dados-2024-02: 1 commits, +0 / -0 LOC


In [8]:
output_dir = "graficosCommits"
os.makedirs(output_dir, exist_ok=True)

print("\n📊 Gerando gráficos por linguagem...")

for lang, repos in top5ReposLanguage.items():
    nomes_repos = []
    commits = []

    for repo in repos:
        nomes_repos.append(repo['name'])
        commits.append(repo['commits'])

    plt.figure(figsize=(10, 6))

    bar_width = 0.4
    x_pos = range(len(nomes_repos))

    bars = plt.bar(x_pos, commits, color='skyblue', width=bar_width)

    plt.xticks(x_pos, nomes_repos, rotation=45, ha='right')
    plt.xlim(-0.5, len(nomes_repos) - 0.5)

    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width() / 2., height,
                 f'{int(height)}',
                 ha='center', va='bottom')

    plt.title(f"Commits por repositório - {lang}")
    plt.xlabel("Repositórios")
    plt.ylabel("Commits")
    plt.tight_layout()

    filepath = os.path.join(output_dir, f"{lang}_commits.png")
    plt.savefig(filepath)
    plt.close()

    print(f"✅ Gráfico salvo: {filepath}")



📊 Gerando gráficos por linguagem...
✅ Gráfico salvo: graficosCommits/Jupyter Notebook_commits.png
✅ Gráfico salvo: graficosCommits/Python_commits.png
✅ Gráfico salvo: graficosCommits/C++_commits.png
✅ Gráfico salvo: graficosCommits/C_commits.png


## **Clonando repositórios das top 5 linguagens e fazendo contagem de locAdd e locRemov de cada um e gerando graficos respectivos para cada linguagem.**

In [9]:
CLONE_DIR = "gitClones"

g = Github(GITHUB_TOKEN)
user = g.get_user(USERNAME)

print(f"\n📛 Nome: {user.name}")
print(f"🔐 Login: {user.login}")
print(f"🖼️ Foto de perfil: {user.avatar_url}\n")

fotoPerfil = user.avatar_url
languageCommitCounter = Counter()
reposLanguage = defaultdict(list)

for repo in user.get_repos():
    try:
        commits = repo.get_commits()
        commit_count = commits.totalCount
        langs = repo.get_languages()
        clone_url = repo.clone_url
        repo_name = repo.name
        repo_path = os.path.join(CLONE_DIR, repo_name)

        for lang in langs:
            languageCommitCounter[lang] += commit_count
            reposLanguage[lang].append({
                'name': repo_name,
                'commits': commit_count,
                'url': clone_url
            })

    except Exception as e:
        print(f"⚠️ Erro ao acessar {repo.name}: {e}")

top_5_languages = languageCommitCounter.most_common(5)
top5ReposLanguage = defaultdict(list)

print("\n🔁 Clonando repositórios e analisando com PyDriller...")

os.makedirs(CLONE_DIR, exist_ok=True)

for lang, _ in top_5_languages:
    for repo_info in reposLanguage[lang]:
        repo_name = repo_info['name']
        commit_count = repo_info['commits']
        url = repo_info['url']
        local_path = os.path.join(CLONE_DIR, repo_name)

        if not os.path.exists(local_path):
            try:
                print(f"🔽 Clonando {repo_name}...")
                git.Repo.clone_from(url, local_path)
            except Exception as e:
                print(f"❌ Falha ao clonar {repo_name}: {e}")
                continue

        loc_add = 0
        loc_remov = 0

        try:
            for commit in Repository(local_path).traverse_commits():
                for mod in commit.modified_files:
                    loc_add += mod.added_lines
                    loc_remov += mod.deleted_lines
        except Exception as e:
            print(f"❌ Erro analisando {repo_name}: {e}")
            continue

        top5ReposLanguage[lang].append({
            'name': repo_name,
            'commits': commit_count,
            'locAdd': loc_add,
            'locRemov': loc_remov
        })

print("\n📂 Repositórios por linguagem com commits e LOC:")
for lang, repos in top5ReposLanguage.items():
    print(f"\n🗂️ {lang}:")
    for repo in repos:
        print(f"   - {repo['name']}: {repo['commits']} commits, +{repo['locAdd']} / -{repo['locRemov']} LOC")



📛 Nome: Enzo Augusto Valentim
🔐 Login: enzoavalentim
🖼️ Foto de perfil: https://avatars.githubusercontent.com/u/163206784?v=4


🔁 Clonando repositórios e analisando com PyDriller...
🔽 Clonando Analise-de-dados-agricolas-Disciplina-Estatistica-2024-02...
🔽 Clonando CurriculoGitHub...
🔽 Clonando Estudos-Iniciais-Em-Python...
🔽 Clonando Trabalhos-Praticos-Programacao-de-Computadores-e-Resolucao-de-Problemas---Python-2023-02...
🔽 Clonando Trabalhos-POO-2024-01...
🔽 Clonando Estrurua-de-dados-2024-02...

📂 Repositórios por linguagem com commits e LOC:

🗂️ Jupyter Notebook:
   - Analise-de-dados-agricolas-Disciplina-Estatistica-2024-02: 2 commits, +3633 / -0 LOC
   - CurriculoGitHub: 6 commits, +712 / -161 LOC

🗂️ Python:
   - Estudos-Iniciais-Em-Python: 2 commits, +846 / -0 LOC
   - Trabalhos-Praticos-Programacao-de-Computadores-e-Resolucao-de-Problemas---Python-2023-02: 1 commits, +254 / -0 LOC

🗂️ Java:
   - Trabalhos-POO-2024-01: 2 commits, +19132 / -0 LOC

🗂️ C++:
   - Estrurua-de-dado

In [10]:
output_dir = "graficosLoc"
os.makedirs(output_dir, exist_ok=True)

print("\n📊 Gerando gráficos de LOC por linguagem...")

for lang, repos in top5ReposLanguage.items():
    nomes_repos = []
    loc_totais = []

    for repo in repos:
        nomes_repos.append(repo['name'])
        total_loc = repo['locAdd'] + repo['locRemov']
        loc_totais.append(total_loc)

    plt.figure(figsize=(10, 6))

    bar_width = 0.4
    x_pos = range(len(nomes_repos))

    bars = plt.bar(x_pos, loc_totais, color='lightcoral', width=bar_width)

    plt.xticks(x_pos, nomes_repos, rotation=45, ha='right')
    plt.xlim(-0.5, len(nomes_repos) - 0.5)

    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width() / 2., height,
                 f'{int(height)}',
                 ha='center', va='bottom')

    plt.title(f"LOC por repositório - {lang}")
    plt.xlabel("Repositórios")
    plt.ylabel("Linhas de Código (LOC)")
    plt.tight_layout()

    filepath = os.path.join(output_dir, f"{lang}_loc.png")
    plt.savefig(filepath)
    plt.close()

    print(f"✅ Gráfico salvo: {filepath}")



📊 Gerando gráficos de LOC por linguagem...
✅ Gráfico salvo: graficosLoc/Jupyter Notebook_loc.png
✅ Gráfico salvo: graficosLoc/Python_loc.png
✅ Gráfico salvo: graficosLoc/Java_loc.png
✅ Gráfico salvo: graficosLoc/C++_loc.png
✅ Gráfico salvo: graficosLoc/C_loc.png


## **Buscando para cada linguagem os top 10 imports feitos.**

In [26]:
import os
import re
from collections import Counter
from pydriller import Repository

LANGUAGE_IMPORT_PATTERNS = {
    'Java': [r'\bimport\s+[\w.*]+;?'],
    'Python': [r'\bimport\s+\w+', r'\bfrom\s+\w+\s+import\s+\w+'],
    'C': [r'#include\s+[<"].+[>"]'],
    'C++': [r'#include\s+[<"].+[>"]', r'\busing\s+namespace\s+\w+'],
    'JavaScript': [r'require\s*\([^)]+\)'],
    'TypeScript': [r'import\s+[^;]+'],
    'PHP': [r'\brequire\s+[^\s;]+', r'\buse\s+[^\s;]+'],
    'Rust': [r'\buse\s+[^\s;]+'],
    'Jupyter Notebook': [r'import\s+\w+', r'from\s+\w+\s+import\s+\w+'],
}


repos_base = "gitClones"
top10ImportsLanguage = {lang: Counter() for lang in top5ReposLanguage.keys()}

repo_to_lang = {}
for lang, repos in top5ReposLanguage.items():
    for repo in repos:
        repo_to_lang[repo['name']] = lang

def extract_imports(text, lang):
    """Aplica os padrões de importação específicos por linguagem."""
    patterns = LANGUAGE_IMPORT_PATTERNS.get(lang, [])
    combined = re.compile('|'.join(patterns))
    return combined.findall(text)

for repo_name in os.listdir(repos_base):
    repo_path = os.path.join(repos_base, repo_name)

    if not os.path.isdir(repo_path):
        continue

    lang = repo_to_lang.get(repo_name)
    if not lang:
        continue 

    print(f"🔍 Processando repositório: {repo_name} ({lang})")

    try:
        for commit in Repository(repo_path).traverse_commits():
            for file in commit.modified_files:
                code_to_scan = file.source_code or file.diff
                if code_to_scan:
                    found_imports = extract_imports(code_to_scan, lang)
                    top10ImportsLanguage[lang].update(found_imports)
    except Exception as e:
        print(f"⚠️ Erro ao processar {repo_name}: {e}")

for lang in top10ImportsLanguage:
    top10ImportsLanguage[lang] = dict(top10ImportsLanguage[lang].most_common(10))


print("\n📦 Top 10 Imports por Linguagem:")
for lang, imports in top10ImportsLanguage.items():
    print(f"\n🔤 {lang}:")
    if imports:
        for imp, count in imports.items():
            print(f"   - {imp}: {count}")
    else:
        print("   (Nenhum import encontrado)")


🔍 Processando repositório: Estudos-Iniciais-Em-Python (Python)
🔍 Processando repositório: Trabalhos-POO-2024-01 (Java)
🔍 Processando repositório: CurriculoGitHub (Jupyter Notebook)
🔍 Processando repositório: Analise-de-dados-agricolas-Disciplina-Estatistica-2024-02 (Jupyter Notebook)
🔍 Processando repositório: Trabalhos-Praticos-Programacao-de-Computadores-e-Resolucao-de-Problemas---Python-2023-02 (Python)
🔍 Processando repositório: Estrurua-de-dados-2024-02 (C)

📦 Top 10 Imports por Linguagem:

🔤 Jupyter Notebook:
   - import matplotlib: 33
   - import pandas: 33
   - from github import Github: 6
   - from collections import Counter: 6
   - import os: 5
   - import numpy: 3
   - from pydriller import Repository: 2
   - import git: 2
   - import shutil: 2
   - import seaborn: 1

🔤 Python:
   - import random: 5
   - import math: 2
   - import time: 2
   - from operator import itemgetter: 1
   - import sys: 1
   - import datetime: 1

🔤 Java:
   - import java.util.ArrayList;: 23
   - impo

In [31]:
print(languageCommitCounter) #Lingugem e numero de commits
print(top5ReposLanguage) #Top5 linguagens com mais commits e repositórios respectivos com linhas add e remov
print(top10ImportsLanguage) #Top 10 bibliotecas com mais imports por linguagem
print(fotoPerfil)
print("/-" * 40)
print("\n📦 Estrutura detalhada de top5ReposLanguage:\n")

for lang, repos in top5ReposLanguage.items():
    print(f"Linguagem: {lang}")
    for repo in repos:
        print(f"   🔸 Repositório: {repo['name']}")
        print(f"      Quantidade commits (commits): {repo['commits']}")
        print(f"      Linhas adicionadas (locAdd): {repo['locAdd']}")
        print(f"      Linhas removidas (locRemov): {repo['locRemov']}")
    print("-" * 40)


Counter({'Jupyter Notebook': 8, 'Python': 3, 'Java': 2, 'C++': 1, 'C': 1, 'HTML': 1})
defaultdict(<class 'list'>, {'Jupyter Notebook': [{'name': 'Analise-de-dados-agricolas-Disciplina-Estatistica-2024-02', 'commits': 2, 'locAdd': 3633, 'locRemov': 0}, {'name': 'CurriculoGitHub', 'commits': 6, 'locAdd': 712, 'locRemov': 161}], 'Python': [{'name': 'Estudos-Iniciais-Em-Python', 'commits': 2, 'locAdd': 846, 'locRemov': 0}, {'name': 'Trabalhos-Praticos-Programacao-de-Computadores-e-Resolucao-de-Problemas---Python-2023-02', 'commits': 1, 'locAdd': 254, 'locRemov': 0}], 'Java': [{'name': 'Trabalhos-POO-2024-01', 'commits': 2, 'locAdd': 19132, 'locRemov': 0}], 'C++': [{'name': 'Estrurua-de-dados-2024-02', 'commits': 1, 'locAdd': 2159, 'locRemov': 0}], 'C': [{'name': 'Estrurua-de-dados-2024-02', 'commits': 1, 'locAdd': 2159, 'locRemov': 0}]})
{'Jupyter Notebook': {'import matplotlib': 33, 'import pandas': 33, 'from github import Github': 6, 'from collections import Counter': 6, 'import os': 5, 